In [1]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [2]:
resorts = pd.read_csv(r"C:\Users\My Computer\Desktop\Dash\resorts.csv", encoding="ISO-8859-1")

In [3]:
resorts.head()

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,...,Snow cannons,Surface lifts,Chair lifts,Gondola lifts,Total lifts,Lift capacity,Child friendly,Snowparks,Nightskiing,Summer skiing
0,1,Hemsedal,60.928244,8.383487,Norway,Europe,46,November - May,1450,620,...,325,15,6,0,21,22921,Yes,Yes,Yes,No
1,2,Geilosiden Geilo,60.534526,8.206372,Norway,Europe,44,November - April,1178,800,...,100,18,6,0,24,14225,Yes,Yes,Yes,No
2,3,Golm,47.057810,9.828167,Austria,Europe,48,December - April,2110,650,...,123,4,4,3,11,16240,Yes,No,No,No
3,4,Red Mountain Resort-Rossland,49.105520,-117.846280,Canada,North America,60,December - April,2075,1185,...,0,2,5,1,8,9200,Yes,Yes,Yes,No
4,5,Hafjell,61.230369,10.529014,Norway,Europe,45,November - April,1030,195,...,150,14,3,1,18,21060,Yes,Yes,Yes,No


In [4]:
print(resorts.isnull().sum())


ID                     0
Resort                 0
Latitude               0
Longitude              0
Country                0
Continent              0
Price                  0
Season                 0
Highest point          0
Lowest point           0
Beginner slopes        0
Intermediate slopes    0
Difficult slopes       0
Total slopes           0
Longest run            0
Snow cannons           0
Surface lifts          0
Chair lifts            0
Gondola lifts          0
Total lifts            0
Lift capacity          0
Child friendly         0
Snowparks              0
Nightskiing            0
Summer skiing          0
dtype: int64


In [8]:
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1(id="title"),
    dcc.Slider(id="price-slider", min=0, max=150, step=10, value=150),
    dcc.RadioItems(
        id="night-ski-toggle",
        options=[
            {"label": "Has Night Skiing", "value": "Yes"},
            {"label": "No Night Skiing", "value": "No"}
        ],
        value="No"
    ),
    dcc.Graph(id="resort-map"),
])

@app.callback(
    Output("title", "children"),
    Output("resort-map", "figure"),
    Input("price-slider", "value"),
    Input("night-ski-toggle", "value")
)
def update_map(price, night_ski):
    filtered_data = resorts[(resorts["Price"] <= price) & (resorts["Nightskiing"] == night_ski)]
    if filtered_data.empty:
        return (
            f"No resorts found with price <= {price} and Night Skiing: {night_ski}",
            px.scatter_mapbox(lat=[0], lon=[0], mapbox_style="open-street-map")
        )
    return (
        f"Resorts with price <= {price} and Night Skiing: {night_ski}",
        px.density_mapbox(
            filtered_data,
            lat='Latitude',
            lon='Longitude',
            z='Total slopes',
            radius=10,
            center=dict(lat=filtered_data['Latitude'].mean(), lon=filtered_data['Longitude'].mean()),
            zoom=5,
            mapbox_style="open-street-map"  # Corrected parameter name
        )
    )

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)

In [33]:
app2 = Dash(__name__)

In [34]:
app2.layout = html.Div([
    html.H1(id="title"),
    dcc.Checklist(
        id="country-checklist",
        options=[
            {"label": "Canada", "value": "Canada"},
            {"label": "United States", "value": "United States"}
        ],
        value=["Canada", "United States"],  # Default selected values
    ),
    dcc.Dropdown(
        id="column-picker",
        options=[{"label": col, "value": col} for col in resorts.select_dtypes("number").columns],
        value="Price"
    ),
    dcc.RadioItems(
        id="ascending-selector",
        options=[
            {"label": "Ascending", "value": True},
            {"label": "Descending", "value": False}
        ],
        value=False,
    ),
    dcc.Graph(id="metric-bar"),
], style={"width": 1200})


In [35]:
@app2.callback(
    Output("title", "children"),
    Output("metric-bar", "figure"),
    Input("country-checklist", "value"),
    Input("column-picker", "value"),
    Input("ascending-selector", "value")
)
def snow_ap(country, column, ascending):
    if not country:
        raise PreventUpdate
    title = f"Top 10 resorts by {column}"

    df = resorts.query("Country in @country")

    fig = px.bar(
        df.sort_values(column, ascending=ascending).iloc[:10],
        x="Resort",
        y=column,
        color="Country",  # Color by country for visual distinction
        width=1000,
        height=800
    )
    return title, fig


if __name__ == "__main__":
    app2.run_server(debug=True, port=8666)